In [1]:
import numpy as np 
import pandas as pd

df = pd.read_csv('heart.csv')
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [2]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [8]:
x = df[['chol','trestbps', 'thalach']]
x

,chol,trestbps,thalach
0,233,145,150
1,250,130,187
2,204,130,172
3,236,120,178
4,354,120,163
...,...,...,...
298,241,140,123
299,264,110,132
300,193,144,141
301,131,130,115


In [9]:
y = df['target']
y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [19]:
import math

In [24]:
import collections

In [25]:
def pre_prob(y):
    y_dict = collections.Counter(y)
    pre_probab = np.ones(2)
    for i in range(0, 2):
        pre_probab[i] = y_dict[i]/y.shape[0]
    return pre_probab

In [26]:
def mean_var(X, y):
    n_features = X.shape[1]
    m = np.ones((2, n_features))
    v = np.ones((2, n_features))
    n_0 = np.bincount(y)[np.nonzero(np.bincount(y))[0]][0]
    x0 = np.ones((n_0, n_features))
    x1 = np.ones((X.shape[0] - n_0, n_features))
    
    k = 0
    for i in range(0, X.shape[0]):
        if y[i] == 0:
            x0[k] = X[i]
            k = k + 1
    k = 0
    for i in range(0, X.shape[0]):
        if y[i] == 1:
            x1[k] = X[i]
            k = k + 1
        
    for j in range(0, n_features):
        m[0][j] = np.mean(x0.T[j])
        v[0][j] = np.var(x0.T[j])*(n_0/(n_0 - 1))
        m[1][j] = np.mean(x1.T[j])
        v[1][j] = np.var(x1.T[j])*((X.shape[0]-n_0)/((X.shape[0]
                                                      - n_0) - 1))
    return m, v # mean and variance 

In [34]:
def prob_feature_class(m, v, x):
    n_features = m.shape[1]
    pfc = np.ones(2)
    for i in range(0, 2):
        product = 1
        for j in range(0, n_features):
            product = product * (1/math.sqrt(2*3.14*v[i][j])) * math.exp(-0.5
                                 * pow((x[j] - m[i][j]),2)/v[i][j])
        pfc[i] = product
    return pfc

In [28]:
def GNB(X, y, x):
    m, v = mean_var(X, y)
    pfc = prob_feature_class(m, v, x)
    pre_probab = pre_prob(y)
    pcf = np.ones(2)
    total_prob = 0
    for i in range(0, 2):
        total_prob = total_prob + (pfc[i] * pre_probab[i])
    for i in range(0, 2):
        pcf[i] = (pfc[i] * pre_probab[i])/total_prob
    prediction = int(pcf.argmax())
    return prediction

In [37]:
X_train = np.array(x_train)
X_train

array([[295, 120, 162],
       [270, 150, 111],
       [231, 150, 147],
       [250, 135, 161],
       [230, 117, 160],
       [303, 128, 159],
       [288, 132, 159],
       [294, 140, 153],
       [335, 140, 158],
       [198, 105, 168],
       [226, 140, 178],
       [244, 150, 154],
       [222, 122, 186],
       [212, 145, 132],
       [262, 130, 155],
       [232, 154, 164],
       [218, 126, 134],
       [254, 125, 163],
       [220, 125, 144],
       [175, 138, 173],
       [184, 132, 105],
       [265, 102, 122],
       [222, 100, 143],
       [211, 110, 161],
       [268, 140, 160],
       [298, 140, 122],
       [239, 140, 151],
       [223, 106, 142],
       [240, 105, 154],
       [197, 140, 116],
       [199,  94, 179],
       [204, 130, 202],
       [219, 120, 158],
       [239, 140, 160],
       [175, 138, 173],
       [197, 130, 131],
       [233, 108, 147],
       [220, 120, 170],
       [309, 125, 131],
       [232, 150, 165],
       [340, 120, 172],
       [204, 112

In [38]:
Y_train = np.array(y_train)
y_train

132    1
202    0
196    0
75     1
176    0
      ..
188    0
71     1
106    1
270    0
102    1
Name: target, Length: 242, dtype: int64

In [41]:
pred = []
X_test = np.array(x_test)
for i in X_test :
    pred.append(GNB(X_train, Y_train, i))
    
pred

[0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0]

In [42]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [44]:
print('f1_score', f1_score(y_test, pred, average="macro"))
print('precision_score', precision_score(y_test, pred, average="macro"))
print('recall_score', recall_score(y_test, pred, average="macro"))

f1_score 0.7516960651289011
precision_score 0.756043956043956
recall_score 0.7510775862068966


In [48]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)

In [49]:
print('f1_score', f1_score(y_test, y_pred, average="macro"))
print('precision_score', precision_score(y_test, y_pred, average="macro"))
print('recall_score', recall_score(y_test, y_pred, average="macro"))

f1_score 0.7516960651289011
precision_score 0.756043956043956
recall_score 0.7510775862068966
